In [1]:
#IMDB review sentiment analysis with pre trained glove

import nltk

from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
#sen = nltk.sent_tokenize(paragraph)
#words = nltk.word_tokenize(paragraph)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [2]:
#read the data
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

dataset = pd.read_csv("/content/drive/My Drive/Colab Notebooks/IMDB Dataset.csv")
# dataset = pd.read_csv('heart1.csv')
dataset

Mounted at /content/drive


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [3]:
data = dataset.iloc[:5000,:]

In [4]:
#text preprocessing
#lowering case, remove stop words,punctuations and apply stemming or lemmatizing (technqiues to find root word)
#stemming - stems or cuts last few char. and might not represtn a meaningful word
#lemmatize - it takes context of word and convets to a meaningful word.helpful in chatbots(time taking process)
#from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import re
corpus = []
ps= PorterStemmer()
removelist=""
for i in range(len(data['review'])):

  review1 =re.sub('','',data['review'][i])          #remove HTML tags
  review2= re.sub('https://.*','',review1)   #remove URLs
  review3 = re.sub('[^a-zA-Z]', ' ',review2)
  review = review3.lower()
  review = review.split()
  reivew = [ps.stem(word) for word in review if word not in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

corpus[4]


'petter mattei s love in the time of money is a visually stunning film to watch mr mattei offers us a vivid portrait about human relations this is a movie that seems to be telling us what money power and success do to people in the different situations we encounter br br this being a variation on the arthur schnitzler s play about the same theme the director transfers the action to the present time new york where all these different characters meet and connect each one is connected in one way or another to the next person but no one seems to know the previous point of contact stylishly the film has a sophisticated luxurious look we are taken to see how these people live and the world they live in their own habitat br br the only thing one gets out of all these souls in the picture is the different stages of loneliness each one inhabits a big city is not exactly the best place in which human relations find sincere fulfillment as one discerns is the case with most of the people we encoun

In [5]:
#
data['sentiment'] = data['sentiment'].map({'positive' :1 ,'negative':0})


<ipython-input-5-d492689735e8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sentiment'] = data['sentiment'].map({'positive' :1 ,'negative':0})


In [6]:
#tokenizer -converts words into tokens in seqeunce order
from keras.preprocessing.text import Tokenizer

tok = Tokenizer()
tok.fit_on_texts(corpus)
token = tok.texts_to_sequences(corpus)

In [7]:
tok.word_index['aamir']

15475

In [9]:
import gensim.downloader as api
embeddings_ap = api.load('glove-wiki-gigaword-100')

[==================================================] 100.0% 128.1/128.1MB downloaded


In [10]:
tok.word_index.items()

dict_items([('the', 1), ('a', 2), ('and', 3), ('of', 4), ('to', 5), ('is', 6), ('br', 7), ('it', 8), ('in', 9), ('i', 10), ('this', 11), ('that', 12), ('s', 13), ('was', 14), ('as', 15), ('movie', 16), ('with', 17), ('for', 18), ('but', 19), ('film', 20), ('you', 21), ('on', 22), ('t', 23), ('not', 24), ('are', 25), ('his', 26), ('he', 27), ('have', 28), ('be', 29), ('one', 30), ('at', 31), ('all', 32), ('they', 33), ('by', 34), ('an', 35), ('who', 36), ('so', 37), ('from', 38), ('like', 39), ('there', 40), ('or', 41), ('just', 42), ('about', 43), ('her', 44), ('if', 45), ('out', 46), ('what', 47), ('has', 48), ('some', 49), ('good', 50), ('can', 51), ('when', 52), ('very', 53), ('more', 54), ('she', 55), ('up', 56), ('no', 57), ('would', 58), ('even', 59), ('time', 60), ('my', 61), ('which', 62), ('story', 63), ('see', 64), ('their', 65), ('really', 66), ('only', 67), ('had', 68), ('well', 69), ('were', 70), ('we', 71), ('me', 72), ('much', 73), ('been', 74), ('than', 75), ('get', 76)

In [14]:
#create embedding matrix
import numpy as np
vocab_len = len(tok.word_index) + 1
embedding_matrix = np.zeros((vocab_len,100))

for (word,i) in tok.word_index.items():
  try:
    word_emb = embeddings_ap[word]
    if word_emb is not None:
      embedding_matrix[i] = word_emb

  except:
    pass





In [15]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.038194  , -0.24487001,  0.72812003, ..., -0.1459    ,
         0.82779998,  0.27061999],
       [-0.27085999,  0.044006  , -0.02026   , ..., -0.4923    ,
         0.63687003,  0.23642001],
       ...,
       [ 0.51052999,  1.2471    , -0.0028446 , ...,  0.49430999,
         0.14184   , -0.43182001],
       [ 0.31678   , -0.37393001, -0.42025   , ..., -0.29802001,
        -0.52503997,  0.69885999],
       [-0.27110001,  0.27258   ,  0.32710001, ...,  0.18047   ,
        -0.19541   , -0.75493002]])

In [16]:
#padding
from keras.preprocessing.sequence import pad_sequences

padded = pad_sequences(token,padding = 'post',maxlen =939 )

In [18]:
#busilding LSTM MODEL
from keras.layers import LSTM,Dense,Bidirectional,Embedding
from keras.models import Sequential
model = Sequential()
model.add(Embedding(input_dim = vocab_len ,weights = [embedding_matrix],output_dim = 100,input_length = 939))
model.add(Bidirectional(LSTM(64)))
#lstm_model.add(Dropout(0.3))
model.add(Dense(1, activation = "sigmoid"))

In [19]:
model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 939, 100)          3849700   
                                                                 
 bidirectional (Bidirection  (None, 128)               84480     
 al)                                                             
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 3934309 (15.01 MB)
Trainable params: 3934309 (15.01 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
y = pd.DataFrame(data['sentiment'])
y.head()

,sentiment
0,1
1,1
2,1
3,0
4,1


In [21]:
model.fit(padded,y,epochs = 5, validation_split=0.3, batch_size=16)

Epoch 1/5
219/219 [==============================] - 276s 1s/step - loss: 0.6669 - accuracy: 0.5869 - val_loss: 0.5917 - val_accuracy: 0.6887
Epoch 2/5
219/219 [==============================] - 267s 1s/step - loss: 0.5134 - accuracy: 0.7523 - val_loss: 0.5142 - val_accuracy: 0.7687
Epoch 3/5
219/219 [==============================] - 262s 1s/step - loss: 0.2834 - accuracy: 0.8914 - val_loss: 0.4390 - val_accuracy: 0.8073
Epoch 4/5
219/219 [==============================] - 266s 1s/step - loss: 0.1108 - accuracy: 0.9669 - val_loss: 0.5564 - val_accuracy: 0.8340
Epoch 5/5
219/219 [==============================] - 264s 1s/step - loss: 0.0355 - accuracy: 0.9909 - val_loss: 0.5835 - val_accuracy: 0.8273


In [27]:
#test our model
sentence = ["The movie was very touching and heart whelming",
            "I have never seen a terrible movie like this",
            "This is the absolute worst film I have ever seen don't waste your time with it "]
# convert to a sequence
sequences = tok.texts_to_sequences(sentence)
# pad the sequence
padded = pad_sequences(sequences, padding='post', maxlen=939)
# Get labels based on probability 1 if p>= 0.5 else 0
prediction = model.predict(padded)
prediction


1/1 [==============================] - 0s 115ms/step


array([[0.9862642 ],
       [0.01283249],
       [0.00203671]], dtype=float32)